In [8]:
import pandas as pd

################################################################################
# DATA LOADING
################################################################################

# Load the main dataset and reference dataset
df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\Matching_WebScrape\4_6.csv')
df2 = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\Matching_WebScrape\4_7.csv')

print(f"Loaded df: {df.shape}")
print(f"Loaded df2: {df2.shape}")

# Display basic info
print(f"\nRows with 'No matches found': {len(df[df['Flag_Reason'] == 'No matches found'])}")
print(f"Exit type rows: {len(df[df['OCR_Address_Type'] == 'Exit'])}")
print(f"Target states (CA, UT, NV, AZ): {len(df[df['OCR_state'].isin(['CA', 'UT', 'NV', 'AZ'])])}")

C:\Users\clint\AppData\Local\Temp\ipykernel_27912\3405881151.py:8: DtypeWarning: Columns (19,36,53,57,72,74,77,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\Matching_WebScrape\4_6.csv')


Loaded df: (67021, 92)
Loaded df2: (15526, 39)

Rows with 'No matches found': 8929
Exit type rows: 35516
Target states (CA, UT, NV, AZ): 4107


In [9]:
#limit df to states of interest
#df = df[df['OCR_state'].isin(['CA', 'UT', 'NV', 'AZ'])]
df


,OCR_Unnamed: 0,OCR_filename,OCR_record_num,OCR_clean_line1,OCR_clean_line2,OCR_line3,OCR_city,OCR_zip_code,OCR_label,OCR_phone,...,Flag_Reason,OCR_address_standardized_ON_parenthesis,OCR_address_standardized_OFF_parenthesis,OCR_Address_Type,OCR_Exit_Number,Exit_From_Address,Exit_From_Label,Is_Unclear_OCR,OCR_Main_Road,OCR_Secondary_Road
0,1,RVersFriend2006-050-ocr.csv,3,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,<U+25A1> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,Available in RVer and Trucker,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN
1,1,RVersFriend2006-050-ocr.csv,3,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,<U+25A1> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,Available in RVer and Trucker,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN
2,2,RVersFriend2007-046-ocr.csv,12,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,24 HRS S,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,Available in RVer and Trucker,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN
3,2,RVersFriend2007-046-ocr.csv,12,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,24 HRS S,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,Available in RVer and Trucker,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN
4,3,TF2008_104_117-5-ocr.csv,6,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,HAS 24 SO <U+2610> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,Available in RVer and Trucker,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67016,38131,RVersFriend2007-004-ocr.csv,13,"C Tok , 99780 Village Gas",6 907-883-4660 AK 1 ( MM 1313.2 ),NaN,Tok,99780,Village Gas,907-883-4660,...,No matches found,( MM 1313.2 ),AK 1,empty,NaN,NaN,NaN,False,AK 1,NaN
67017,38132,TF2008_006_021-3-ocr.csv,16,"C Tok , 99780 Village Gas",6 907-883-4660 AK 1 ( MM 1313.2 ),<U+25C9>,Tok,99780,Village Gas,907-883-4660,...,No matches found,( MM 1313.2 ),AK 1,empty,NaN,NaN,NaN,False,AK 1,NaN
67018,38133,RVersFriend2006-007-ocr.csv,15,"Tok , 99780 Plaza Truck Stop ( Texaco )",907-883-5833 AK Hwy 2 ( MM 1313.5 ),NaN,Tok,99780,Plaza Truck Stop ( Texaco ),907-883-5833,...,No matches found,( MM 1313.5 ),AK Hwy 2,empty,NaN,NaN,NaN,False,Hwy 2,NaN
67019,38134,RVersFriend2007-004-ocr.csv,15,"C Tok , 99780 Plaza Truck Stop ( Texaco )",907-883-5833 AK Hwy 2 ( MM 1313.5 ),NaN,Tok,99780,Plaza Truck Stop ( Texaco ),907-883-5833,...,No matches found,( MM 1313.5 ),AK Hwy 2,empty,NaN,NaN,NaN,False,Hwy 2,NaN


In [10]:
################################################################################
# FINAL CLEAN IMPLEMENTATION OF MATCHING LOGIC
################################################################################

import pandas as pd
import numpy as np
import re

def clean_text_for_matching(text):
    """Clean text for case-insensitive matching, removing hyphens, apostrophes, spaces, periods"""
    if pd.isna(text):
        return ""
    return re.sub(r"['-.\s]", "", str(text).lower())

def filter_meaningful_words(text):
    """
    Extract meaningful words for name matching, excluding common generic terms.
    Returns a list of words that are meaningful for matching business names.
    """
    if pd.isna(text):
        return []
    
    # Common words to exclude from matching (case-insensitive)
    exclude_words = {
        # Vehicle/Transportation related
        'truck', 'trucks', 'trucker', 'truckers', 'trucking',
        'stop', 'stops', 'station', 'stations',
        'travel', 'travels', 'traveling', 'centre', 'center',
        'plaza', 'plazas', 'mart', 'store', 'stores',
        'gas', 'fuel', 'petrol', 'petroleum', 'diesel',
        'service', 'services', 'auto', 'automotive',
        'parking', 'fleet', 'commercial', 'professional',
        
        # Generic business terms
        'inc', 'llc', 'corp', 'corporation', 'company', 'co',
        'ltd', 'limited', 'enterprises', 'enterprise',
        'group', 'international', 'intl', 'usa', 'america',
        'franchise', 'chain', 'brand', 'corporate',
        
        # Location/Direction terms (but keep specific place names)
        'north', 'south', 'east', 'west', 'ne', 'nw', 'se', 'sw',
        'highway', 'hwy', 'road', 'rd', 'street', 'st', 'avenue', 'ave',
        'exit', 'interchange', 'junction',
        
        # Common adjectives/descriptors (removed 'town' and 'city' as they can be part of business names)
        'big', 'small', 'large', 'giant', 'super', 'mega', 'express',
        'quick', 'fast', 'convenient', 'friendly', 'family',
        'country', 'state', 'national', 'speedy', 'instant',
        
        # Food/Restaurant terms
        'restaurant', 'cafe', 'diner', 'grill', 'kitchen', 'food', 'eats',
        
        # Accommodation terms
        'motel', 'hotel', 'inn', 'lodge', 'rest',
        
        # Time/Convenience terms
        '24', 'hour', 'hours',
        
        # Connectors and articles
        'and', 'or', 'the', 'of', 'in', 'at', 'on', 'for', 'with', 'by'
    }
    
    # Split text into words and clean each word
    words = str(text).lower().split()
    meaningful_words = []
    
    for word in words:
        # Clean the word (remove punctuation, etc.)
        cleaned_word = re.sub(r"[^\w]", "", word)
        
        # Keep words that are:
        # 1. At least 3 characters long
        # 2. Not in the exclude list
        # 3. Not purely numeric
        if (len(cleaned_word) >= 3 and 
            cleaned_word not in exclude_words and 
            not cleaned_word.isdigit()):
            meaningful_words.append(cleaned_word)
    
    return meaningful_words

def normalize_zip_code(zip_code):
    """Normalize ZIP code to 5 digits with leading zeros (e.g., '2345' -> '02345')"""
    if pd.isna(zip_code):
        return None
    try:
        # Remove any non-digit characters and convert to string
        zip_str = re.sub(r'\D', '', str(zip_code))
        if len(zip_str) >= 5:
            return zip_str[:5]  # Take first 5 digits
        elif len(zip_str) > 0:
            return zip_str.zfill(5)  # Pad with leading zeros to make 5 digits
        else:
            return None
    except:
        return None

def comprehensive_matching_logic():
    """
    Complete implementation of the 14-step matching logic as specified:
    1. Filter rows where Flag_Reason equals "No matches found" and OCR_Address_Type equals "Exit"
    2. Process all filtered rows
    3-8. Progressive matching: ZIP -> State -> Exit -> Highway -> Name
    9-14. Handle different match scenarios and create duplicate rows for multiple matches
    """
    
    # Step 1: Filter rows
    filtered_df = df[(df['Flag_Reason'] == "No matches found") & (df['OCR_Address_Type'] == "Exit")].copy()
    
    if len(filtered_df) == 0:
        print("No rows found matching the filter criteria")
        return df
    
    # Step 2: Process all filtered rows
    selected_rows = filtered_df.copy()
    
    # Create working copy of dataframe - we'll build this incrementally
    result_df = df.copy()
    
    # Initialize "Manually Verified?" column with default value "No"
    if 'Manually Verified?' not in result_df.columns:
        result_df['Manually Verified?'] = "No"
    
    # Keep track of new rows to add
    new_rows_to_add = []
    
    # Prepare df2 with normalized ZIP codes for efficient matching
    df2_work = df2.copy()
    df2_work['ZIP_normalized'] = df2_work['Postal Code'].apply(normalize_zip_code)
    
    # Create mapping from df2 columns to Scraped_ columns
    def map_df2_to_scraped(df2_row):
        """Map a df2 row to scraped column values"""
        scraped_data = {}
        for df2_col in df2.columns:
            scraped_col = f"Scraped_{df2_col}"
            if scraped_col in result_df.columns:
                scraped_data[scraped_col] = df2_row[df2_col]
        return scraped_data
    
    print(f"Processing {len(selected_rows)} rows...")
    
    # Process each selected row (Steps 3-14)
    for row_num, (idx, row) in enumerate(selected_rows.iterrows(), 1):
        print(f"\nRow {row_num}/{len(selected_rows)} (Index {idx}):")
        
        # Step 3: Extract and normalize OCR_zip_code
        ocr_zip = normalize_zip_code(row['OCR_zip_code'])
        
        # Step 4: Check ZIP code match
        if not ocr_zip:
            result_df.loc[idx, 'Flagged'] = True
            result_df.loc[idx, 'Flag_Reason'] = "no matching ZIPCODE"
            print(f"  ❌ No valid ZIP code")
            continue
            
        zip_matches = df2_work[df2_work['ZIP_normalized'] == ocr_zip]
        if zip_matches.empty:
            result_df.loc[idx, 'Flagged'] = True
            result_df.loc[idx, 'Flag_Reason'] = "no matching ZIPCODE"
            print(f"  ❌ ZIP {ocr_zip} not found")
            continue
        
        print(f"  ✓ ZIP {ocr_zip}: {len(zip_matches)} matches")
        
        # Step 5: Check state match
        ocr_state = row['OCR_state']
        state_matches = zip_matches[zip_matches['State'] == ocr_state]
        if state_matches.empty:
            result_df.loc[idx, 'Flagged'] = True
            result_df.loc[idx, 'Flag_Reason'] = "matching ZIPCODE, no matching State"
            print(f"  ❌ State {ocr_state} not found")
            continue
        
        print(f"  ✓ State {ocr_state}: {len(state_matches)} matches")
        
        # Step 6: Check exit number match
        ocr_exit = row['OCR_Exit_Number']
        try:
            ocr_exit_str = str(int(float(ocr_exit))) if pd.notna(ocr_exit) else ""
        except:
            ocr_exit_str = str(ocr_exit).strip() if pd.notna(ocr_exit) else ""
        
        if ocr_exit_str:
            exit_matches = state_matches[
                state_matches['Exit'].astype(str).str.contains(ocr_exit_str, case=False, na=False, regex=False)
            ]
        else:
            exit_matches = pd.DataFrame()
            
        if exit_matches.empty:
            result_df.loc[idx, 'Flagged'] = True
            result_df.loc[idx, 'Flag_Reason'] = "matching ZIPCODE, matching State, no matching Exit"
            print(f"  ❌ Exit {ocr_exit} not found")
            continue
        
        print(f"  ✓ Exit {ocr_exit}: {len(exit_matches)} matches")
        
        # Step 7: Check highway match (UPDATED TO CHECK MULTIPLE COLUMNS)
        ocr_highway = row['OCR_Main_Road']
        
        # Check against Highway, Exit, and Street Address columns
        highway_condition = (
            (exit_matches['Highway'] == ocr_highway) |
            (exit_matches['Exit'] == ocr_highway) |
            (exit_matches['Street Address'] == ocr_highway)
        )
        
        highway_matches = exit_matches[highway_condition]
        
        if highway_matches.empty:
            result_df.loc[idx, 'Flagged'] = True
            result_df.loc[idx, 'Flag_Reason'] = "matching ZIPCODE, matching State, matching Exit, no matching Highway/Exit/Street Address"
            print(f"  ❌ Highway/Exit/Street Address '{ocr_highway}' not found in any of the three columns")
            continue
        
        # Show which column(s) matched for debugging
        highway_match_details = []
        for _, match_row in highway_matches.iterrows():
            matched_cols = []
            if match_row['Highway'] == ocr_highway:
                matched_cols.append('Highway')
            if match_row['Exit'] == ocr_highway:
                matched_cols.append('Exit')
            if match_row['Street Address'] == ocr_highway:
                matched_cols.append('Street Address')
            highway_match_details.append(f"({', '.join(matched_cols)})")
        
        print(f"  ✓ Highway/Exit/Street Address '{ocr_highway}': {len(highway_matches)} matches")
        print(f"    Match details: {', '.join(set(highway_match_details))}")
        
        # Step 8: Check name matches
        ocr_label = row['OCR_label']
        if pd.isna(ocr_label):
            ocr_words = []
        else:
            ocr_words = filter_meaningful_words(ocr_label)
        
        name_match_indices = []
        for hw_idx, hw_row in highway_matches.iterrows():
            chain_words = filter_meaningful_words(hw_row['Chain'])
            name_words = filter_meaningful_words(hw_row['name'])
            business_words = chain_words + name_words
            
            match_found = False
            matched_words = []
            
            for ocr_word in ocr_words:
                for business_word in business_words:
                    if (ocr_word in business_word or business_word in ocr_word or
                        ocr_word == business_word):
                        match_found = True
                        matched_words.append(f"{ocr_word}→{business_word}")
                        break
                if match_found:
                    break
            
            if match_found:
                name_match_indices.append(hw_idx)
                print(f"    🎯 Match found: {matched_words}")
        
        print(f"  🔍 Name matching: '{ocr_label}'")
        print(f"      OCR meaningful words: {ocr_words}")
        print(f"      Total matches found: {len(name_match_indices)}")
        
        # Steps 9-14: Handle different match scenarios and CREATE ROWS FOR EACH MATCH
        if len(name_match_indices) == 0:
            # Step 13: No name matches
            result_df.loc[idx, 'Flagged'] = True
            result_df.loc[idx, 'Flag_Reason'] = "No Phone/Name/ZIP/State/Exit/Highway matches found"
            print(f"  ❌ No name matches found")
            
        else:
            # We have matches - create rows for each match
            print(f"  ✅ Creating {len(name_match_indices)} row(s) for {len(name_match_indices)} match(es)")
            
            for match_num, match_idx in enumerate(name_match_indices):
                match_row = highway_matches.loc[match_idx]
                
                if match_num == 0:
                    # First match: update the original row
                    target_idx = idx
                    result_df.loc[target_idx, 'Flagged'] = False
                    if len(name_match_indices) == 1:
                        result_df.loc[target_idx, 'Flag_Reason'] = "Single Name/ZIP/State/Exit/Highway Match Found"
                        # Set manual verification for single matches
                        result_df.loc[target_idx, 'Manually Verified?'] = "Please Verify This"
                    else:
                        result_df.loc[target_idx, 'Flag_Reason'] = f"Multiple Name/ZIP/State/Exit/Highway Matches Found - Match {match_num + 1} of {len(name_match_indices)}"
                        # Set manual verification for multiple matches
                        result_df.loc[target_idx, 'Manually Verified?'] = "Please Verify This"
                    
                    print(f"    📝 Filling original row with match {match_num + 1}")
                else:
                    # Additional matches: create new rows
                    new_row = result_df.loc[idx].copy()  # Copy the original row
                    new_row['Flagged'] = False
                    new_row['Flag_Reason'] = f"Multiple Name/ZIP/State/Exit/Highway Matches Found - Match {match_num + 1} of {len(name_match_indices)}"
                    # Set manual verification for multiple matches in new rows
                    new_row['Manually Verified?'] = "Please Verify This"
                    
                    # Clear existing scraped data
                    for col in new_row.index:
                        if col.startswith('Scraped_'):
                            new_row[col] = None
                    
                    new_rows_to_add.append(new_row)
                    target_idx = len(new_rows_to_add) - 1  # This will be used for tracking
                    
                    print(f"    ➕ Creating new row for match {match_num + 1}")
                
                # Fill scraped data for this match (either original row or new row)
                scraped_data = map_df2_to_scraped(match_row)
                
                if match_num == 0:
                    # Update original row
                    for col, value in scraped_data.items():
                        result_df.loc[idx, col] = value
                else:
                    # Update the new row we just created
                    for col, value in scraped_data.items():
                        new_rows_to_add[-1][col] = value
                
                print(f"      ✓ Filled data for {match_row['Chain']} - {match_row['name']}")
    
    # Add all new rows to the result dataframe
    if new_rows_to_add:
        new_rows_df = pd.DataFrame(new_rows_to_add)
        result_df = pd.concat([result_df, new_rows_df], ignore_index=True)
        print(f"\n🎉 Added {len(new_rows_to_add)} new rows for multiple matches!")
    
    print(f"\n📊 Final result: {len(result_df)} rows (original: {len(df)}, added: {len(new_rows_to_add)})")
    return result_df

# Execute the final implementation
final_result = comprehensive_matching_logic()

Processing 4220 rows...

Row 1/4220 (Index 150):
  ✓ ZIP 01566: 4 matches
  ✓ State MA: 4 matches
  ❌ Exit 9 not found

Row 2/4220 (Index 151):
  ✓ ZIP 01566: 4 matches
  ✓ State MA: 4 matches
  ❌ Exit 9 not found

Row 3/4220 (Index 152):
  ✓ ZIP 01566: 4 matches
  ✓ State MA: 4 matches
  ❌ Exit 9 not found

Row 4/4220 (Index 216):
  ❌ ZIP 01852 not found

Row 5/4220 (Index 217):
  ❌ ZIP 01852 not found

Row 6/4220 (Index 218):
  ❌ ZIP 01852 not found

Row 7/4220 (Index 219):
  ❌ ZIP 01852 not found

Row 8/4220 (Index 220):
  ❌ ZIP 01852 not found

Row 9/4220 (Index 221):
  ❌ ZIP 01852 not found

Row 10/4220 (Index 354):
  ✓ ZIP 02817: 2 matches
  ✓ State RI: 2 matches
  ✓ Exit 5: 2 matches
  ✓ Highway/Exit/Street Address 'I-95': 2 matches
    Match details: (Highway)
  🔍 Name matching: 'RI's Only 24 Hr A / T Plaza )'
      OCR meaningful words: ['ris', 'only']
      Total matches found: 0
  ❌ No name matches found

Row 11/4220 (Index 355):
  ✓ ZIP 02817: 2 matches
  ✓ State RI: 2 matc

C:\Users\clint\AppData\Local\Temp\ipykernel_27912\1591124546.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[idx, col] = value
C:\Users\clint\AppData\Local\Temp\ipykernel_27912\1591124546.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[idx, col] = value


      ✓ Filled data for shell - CHAMPLAIN FARMS
    ➕ Creating new row for match 2
      ✓ Filled data for shell - CHAMPLAIN FARMS

Row 34/4220 (Index 1332):
  ✓ ZIP 07430: 10 matches
  ✓ State NJ: 10 matches
  ✓ Exit 66: 6 matches
  ✓ Highway/Exit/Street Address 'I-287': 6 matches
    Match details: (Highway)
    🎯 Match found: ['mahwah→mahwah']
    🎯 Match found: ['mahwah→mahwah']
  🔍 Name matching: 'Mahwah Fuel Stop ( Citgo )'
      OCR meaningful words: ['mahwah', 'citgo']
      Total matches found: 2
  ✅ Creating 2 row(s) for 2 match(es)
    📝 Filling original row with match 1
      ✓ Filled data for sunoco - KINGS MAHWAH TRUCK STOP
    ➕ Creating new row for match 2
      ✓ Filled data for sunoco - KINGS MAHWAH TRUCK STOP

Row 35/4220 (Index 1333):
  ✓ ZIP 07430: 10 matches
  ✓ State NJ: 10 matches
  ✓ Exit 66: 6 matches
  ✓ Highway/Exit/Street Address 'I-287': 6 matches
    Match details: (Highway)
    🎯 Match found: ['mahwah→mahwah']
    🎯 Match found: ['mahwah→mahwah']
  🔍 Nam

In [11]:
print(f"Original df shape: {df.shape}")
print(f"Final result shape: {final_result.shape}")
print(f"New rows added: {final_result.shape[0] - df.shape[0]}")

# Show some examples of multiple matches
multiple_matches = final_result[final_result['Flag_Reason'].str.contains('Multiple.*Match.*of', na=False)]
print(f"\nRows with multiple matches: {len(multiple_matches)}")

if len(multiple_matches) > 0:
    print("\nSample of multiple match rows:")
    sample_cols = ['OCR_label', 'Flag_Reason', 'Scraped_Chain', 'Scraped_name', 'Flagged']
    print(multiple_matches[sample_cols].head(10))

final_result

Original df shape: (67021, 92)
Final result shape: (67857, 93)
New rows added: 836

Rows with multiple matches: 1620

Sample of multiple match rows:
                         OCR_label  \
1044     Champlain Farms ( Exxon )   
1045     Champlain Farms ( Exxon )   
1046   Champlain Farms ( Exxon ) )   
1047     Champlain Farms ( Shell )   
1048     Champlain Farms ( Shell )   
1049     Champlain Farms ( Shell )   
1332    Mahwah Fuel Stop ( Citgo )   
1333              Mahwah Fuel Stop   
1334              Mahwah Fuel Stop   
1335  Mahwah Fuel Stop ( Liberty )   

                                            Flag_Reason Scraped_Chain  \
1044  Multiple Name/ZIP/State/Exit/Highway Matches F...         shell   
1045  Multiple Name/ZIP/State/Exit/Highway Matches F...         shell   
1046  Multiple Name/ZIP/State/Exit/Highway Matches F...         shell   
1047  Multiple Name/ZIP/State/Exit/Highway Matches F...         shell   
1048  Multiple Name/ZIP/State/Exit/Highway Matches F...         she

,OCR_Unnamed: 0,OCR_filename,OCR_record_num,OCR_clean_line1,OCR_clean_line2,OCR_line3,OCR_city,OCR_zip_code,OCR_label,OCR_phone,...,OCR_address_standardized_ON_parenthesis,OCR_address_standardized_OFF_parenthesis,OCR_Address_Type,OCR_Exit_Number,Exit_From_Address,Exit_From_Label,Is_Unclear_OCR,OCR_Main_Road,OCR_Secondary_Road,Manually Verified?
0,1,RVersFriend2006-050-ocr.csv,3,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,<U+25A1> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN,No
1,1,RVersFriend2006-050-ocr.csv,3,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,<U+25A1> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN,No
2,2,RVersFriend2007-046-ocr.csv,12,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,24 HRS S,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN,No
3,2,RVersFriend2007-046-ocr.csv,12,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,24 HRS S,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN,No
4,3,TF2008_104_117-5-ocr.csv,6,"Blandford , 01008 Blandford Plaza EB Exxon # 5020",413-848-2056 1-90 ( MATP ) MM 29 EB,HAS 24 SO <U+2610> <U+2610>,Blandford,1008,Blandford Plaza EB Exxon # 5020,413-848-2056,...,( MATP ) MM 29 EB,I-90,empty,NaN,NaN,NaN,False,I-90,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67852,37999,TF2014_200_213-3-ocr.csv,43,Ontario ( 97914 ) Pilot Travel Center # 232,541-889-3580 1-84 Exit 376 A ( US 30 W ),Pilot 24 L,Ontario,97914,Pilot Travel Center # 232,541-889-3580,...,( US 30 W ),I-84 Exit 376 A,Exit,376,376,NaN,False,I-84,NaN,Please Verify This
67853,38000,TF2015_200_215-3-ocr.csv,44,Ontario ( 97914 ) Pilot Travel Center # 232,8 541-889-3580 1-84 Exit 376 A ( US 30 W ) Pilot,24 L,Ontario,97914,Pilot Travel Center # 232,541-889-3580,...,( US 30 W ) Pilot,I-84 Exit 376 A,Exit,376,376,NaN,False,I-84,NaN,Please Verify This
67854,38001,TF2016_194_209-6-ocr.csv,44,Ontario ( 97914 ) Pilot Travel Center # 232,541-889-3580 1-84 Exit 376 A ( US 30 W ) Pilot,Pilot 24 024 0,Ontario,97914,Pilot Travel Center # 232,541-889-3580,...,( US 30 W ) Pilot,I-84 Exit 376 A,Exit,376,376,NaN,False,I-84,NaN,Please Verify This
67855,38103,TF2015_274_289-0-ocr.csv,9,Pasco ( 99301 ) Khalsa King City Truck Stop - ...,509-547-3475 1-182 Exit 14 ( US 395 N - 3/4 ),24 M,Pasco,99301,Khalsa King City Truck Stop - 3/4 ( Shell mi )...,509-547-3475,...,( US 395 N - 3/4 ),I-182 Exit 14,Exit,14,14,NaN,False,I-182,NaN,Please Verify This


In [12]:
# Save the enhanced results to CSV
final_result.to_csv('5.csv', index=False)
print(f"Enhanced results saved to 5.csv")
print(f"✅ Original rows: {df.shape[0]}")
print(f"✅ Final rows: {final_result.shape[0]}")
print(f"✅ Additional rows created for multiple matches: {final_result.shape[0] - df.shape[0]}")
print(f"✅ Total successful matches: {len(final_result[final_result['Flagged'] == False])}")
print("🎉 All multiple matches now have their own dedicated rows!")

Enhanced results saved to 5.csv
✅ Original rows: 67021
✅ Final rows: 67857
✅ Additional rows created for multiple matches: 836
✅ Total successful matches: 59393
🎉 All multiple matches now have their own dedicated rows!
